In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing


In [2]:
## Load the dataset
df = pd.read_csv('../data/dataset/dataset.csv')
df.head()

,1st_team,2nd_team,year,net_score,GK_height_cm_1st,GK_weight_kg_1st,GK_age_1st,GK_overall_1st,GK_potential_1st,GK_attacking_crossing_1st,...,RAM_mentality_vision_2nd,RAM_mentality_penalties_2nd,RAM_defending_marking_awareness_2nd,RAM_defending_standing_tackle_2nd,RAM_defending_sliding_tackle_2nd,RAM_goalkeeping_diving_2nd,RAM_goalkeeping_handling_2nd,RAM_goalkeeping_kicking_2nd,RAM_goalkeeping_positioning_2nd,RAM_goalkeeping_reflexes_2nd
0,Norway,Poland,2014,-3.0,192.0,78.0,23.0,72.0,75.0,25.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Australia,Ecuador,2014,-1.0,182.0,84.0,22.0,73.0,79.0,25.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Austria,Uruguay,2014,0.0,194.0,85.0,30.0,64.0,64.0,25.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Czech Republic,Norway,2014,0.0,196.0,90.0,32.0,85.0,85.0,25.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,England,Denmark,2014,1.0,196.0,91.0,27.0,82.0,83.0,25.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
df.tail()

,1st_team,2nd_team,year,net_score,GK_height_cm_1st,GK_weight_kg_1st,GK_age_1st,GK_overall_1st,GK_potential_1st,GK_attacking_crossing_1st,...,RAM_mentality_vision_2nd,RAM_mentality_penalties_2nd,RAM_defending_marking_awareness_2nd,RAM_defending_standing_tackle_2nd,RAM_defending_sliding_tackle_2nd,RAM_goalkeeping_diving_2nd,RAM_goalkeeping_handling_2nd,RAM_goalkeeping_kicking_2nd,RAM_goalkeeping_positioning_2nd,RAM_goalkeeping_reflexes_2nd
1116,Canada,Uruguay,2022,-2.0,195.0,84.0,34.0,75.0,75.0,15.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1117,Ecuador,Japan,2022,0.0,195.0,81.0,35.0,74.0,74.0,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1118,Iran,Senegal,2022,0.0,194.0,85.0,29.0,74.0,75.0,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1119,Saudi Arabia,United States,2022,0.0,185.0,79.0,30.0,71.0,71.0,14.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1120,Portugal,Spain,2022,-1.0,190.0,84.0,34.0,82.0,82.0,12.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
## Drop the columns that are not needed
df.drop("year", axis=1, inplace=True)

In [5]:
def class_maper(score):
    if score == 0:
        return 0
    elif score > 0:
        return 1
    elif score < 0:
        return 2
    

In [6]:
## Split the dataset into train and test
from sklearn.model_selection import train_test_split

X = df.drop('net_score', axis=1)
y = df['net_score'].apply(class_maper)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [7]:
le = preprocessing.LabelEncoder()

countries_set = set(list(X["2nd_team"]) + list(X["1st_team"]))

countries_encoder = le.fit(list(countries_set))

X_train["2nd_team"] = countries_encoder.transform(X_train["2nd_team"])
X_train["1st_team"] = countries_encoder.transform(X_train["1st_team"])

X_test["2nd_team"] = countries_encoder.transform(X_test["2nd_team"])
X_test["1st_team"] = countries_encoder.transform(X_test["1st_team"])

In [8]:
X_train

,1st_team,2nd_team,GK_height_cm_1st,GK_weight_kg_1st,GK_age_1st,GK_overall_1st,GK_potential_1st,GK_attacking_crossing_1st,GK_attacking_finishing_1st,GK_attacking_heading_accuracy_1st,...,RAM_mentality_vision_2nd,RAM_mentality_penalties_2nd,RAM_defending_marking_awareness_2nd,RAM_defending_standing_tackle_2nd,RAM_defending_sliding_tackle_2nd,RAM_goalkeeping_diving_2nd,RAM_goalkeeping_handling_2nd,RAM_goalkeeping_kicking_2nd,RAM_goalkeeping_positioning_2nd,RAM_goalkeeping_reflexes_2nd
737,31,21,185.0,83.0,30.0,83.0,84.0,17.0,10.0,12.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1027,24,18,191.0,86.0,31.0,85.0,85.0,16.0,10.0,23.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39,4,23,176.0,63.0,38.0,63.0,63.0,25.0,25.0,25.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30,38,23,190.0,85.0,26.0,80.0,84.0,25.0,25.0,25.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
989,21,20,193.0,93.0,35.0,90.0,90.0,15.0,13.0,25.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
466,34,14,190.0,82.0,32.0,82.0,82.0,18.0,13.0,16.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
121,36,35,185.0,84.0,28.0,69.0,69.0,25.0,25.0,25.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1044,27,3,196.0,90.0,22.0,89.0,93.0,12.0,12.0,12.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1095,58,3,191.0,88.0,29.0,76.0,76.0,13.0,11.0,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
## Apply PCA
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

pca = PCA(n_components=0.9)
scaler = StandardScaler()



In [10]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression


estimators_dict = {
RandomForestClassifier():  
{
    "est__n_estimators": [10, 50, 100, 200, 500],
    "est__max_depth": [3, 5, 10, 20, 50, 100, None],
    "est__min_samples_split": [2, 5, 10],
    "est__min_samples_leaf": [1, 2, 4],
    "est__bootstrap": [True, False],
    "est__criterion": ["gini", "entropy"]
},
AdaBoostClassifier():
{
    "est__n_estimators": [10, 50, 100, 200, 500],
    "est__learning_rate": [0.01, 0.05, 0.1, 0.5, 1.0],
    "est__algorithm": ["SAMME", "SAMME.R"]
},
GradientBoostingClassifier():
{
    "est__loss": ["log_loss", "exponential"],
    "est__learning_rate": [0.01, 0.05, 0.1, 0.5, 1.0],
    "est__n_estimators": [10, 50, 100, 200, 500],
    "est__max_depth": [3, 5, 10, 20, 50, 100, None],
    "est__min_samples_split": [2, 5, 10],
    "est__min_samples_leaf": [1, 2, 4],
    "est__criterion": ["friedman_mse", "mse", "mae"]
},
KNeighborsClassifier():
{
    "est__n_neighbors": [3, 5, 10, 20, 50, 100],
    "est__weights": ["uniform", "distance"],
    "est__algorithm": ["auto", "ball_tree", "kd_tree", "brute"],
    "est__leaf_size": [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
},
SVC():
{
    "est__C": [0.1, 0.5, 1.0, 5.0, 10.0, 50.0, 100.0],
    "est__kernel": ["linear", "poly", "rbf", "sigmoid"],
    "est__degree": [2, 3, 4, 5, 6, 7, 8, 9, 10],
    "est__gamma": ["scale", "auto"],
    "est__shrinking": [True, False]
},
LogisticRegression():
{
    "est__penalty": ["l1", "l2", "elasticnet", "none"],
    "est__C": [0.1, 0.5, 1.0, 5.0, 10.0, 50.0, 100.0],
    "est__solver": ["newton-cg", "lbfgs", "liblinear", "sag", "saga"],
    "est__max_iter": [100, 200, 500, 1000, 2000, 5000, 10000]
}
}

In [11]:
from sklearn.model_selection import GridSearchCV

def get_best_estimator(estimator, params, cv):
    grid = GridSearchCV(estimator, params, cv=cv, scoring="neg_mean_squared_error", n_jobs=-1)
    grid.fit(X_train, y_train)
    return grid.best_estimator_


In [12]:
### Get tunned pipeline
from sklearn.model_selection import cross_validate, KFold

scorers = ["f1_micro", "roc_auc"]
train_f1_micro = "train_f1_micro"
test_f1_micro = "test_f1_micro"

train_roc_auc = "train_roc_auc"
test_roc_auc = "test_roc_auc"
cv = KFold(n_splits=5, shuffle=True, random_state=42)

valid_results = pd.DataFrame(columns=["estimator", "params", "train_f1_micro", "test_f1_micro", "train_roc_auc", 
                                      "test_roc_auc"])    

                                      
for est, params in estimators_dict.items():
    print("Getting best estimator for {}".format(est))
    pipeline = Pipeline(steps=[('scaler', scaler), ('pca', pca), ('est', est)])
    tunned_estimator = get_best_estimator(pipeline, params, cv)

    
    scores = cross_validate(tunned_estimator, X_train, y_train, cv=cv, return_train_score=True, scoring=scorers, n_jobs=-1)
    valid_results = valid_results.append({
        "estimator": est,
        "params": tunned_estimator.get_params(),
        train_f1_micro: scores["train_f1_micro"].mean(),
        test_f1_micro: scores["test_f1_micro"].mean(),
        train_roc_auc: scores["train_roc_auc"].mean(),
        test_roc_auc: scores["test_roc_auc"].mean()
    }, ignore_index=True)

valid_results

Getting best estimator for RandomForestClassifier()


/Users/zghazal/opt/miniconda3/envs/fifa_env/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/zghazal/opt/miniconda3/envs/fifa_env/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/zghazal/opt/miniconda3/envs/fifa_env/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 106, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/Users/zghazal/opt/miniconda3/envs/fifa_env/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 352, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
/Users/zghazal/opt/miniconda3/envs/fifa_env/lib/python3.8/site-packages/sklearn/mod

Getting best estimator for AdaBoostClassifier()


/Users/zghazal/opt/miniconda3/envs/fifa_env/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/zghazal/opt/miniconda3/envs/fifa_env/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/zghazal/opt/miniconda3/envs/fifa_env/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 106, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/Users/zghazal/opt/miniconda3/envs/fifa_env/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 352, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
/Users/zghazal/opt/miniconda3/envs/fifa_env/lib/python3.8/site-packages/sklearn/mod

Getting best estimator for GradientBoostingClassifier()


/Users/zghazal/opt/miniconda3/envs/fifa_env/lib/python3.8/site-packages/sklearn/ensemble/_gb.py:310: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(
/Users/zghazal/opt/miniconda3/envs/fifa_env/lib/python3.8/site-packages/sklearn/ensemble/_gb.py:310: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(
/Users/zghazal/opt/miniconda3/envs/fifa_env/lib/python3.8/site-packages/sklearn/ensemble/_gb.py:310: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(
/Users/zghazal/opt/miniconda3/envs/fifa_env/lib/python3.8/site-packages/sklearn/ensemble/_gb.py:310: FutureWarning: The loss parameter name 'd

In [ ]:
valid_results

array([ 0.07775609, -0.19959367, -0.24583849,  0.05017579, -0.55332422,
       -0.25569056, -0.72920754,  1.09212296, -0.54988223,  0.07815927,
       -0.59048087, -0.28424636,  0.09101597,  0.01541676,  0.95909748,
       -0.60226392, -0.42897447,  0.01535695,  1.27148602,  1.11903654,
        0.73226245,  1.17818262, -0.19083547, -0.2053676 ,  1.01596918,
       -0.23241941, -0.23378492,  0.95685552,  0.17084135,  0.19344579,
        0.92101156, -0.71596833, -0.62665391,  0.16460455,  0.94781815,
        0.1029961 ,  0.97110533, -0.20639189,  1.28401084,  0.49705126,
        0.88097078,  0.79671906,  0.5884058 ,  0.1460685 ,  0.18362757,
        0.07436703,  1.25124983,  0.50226441,  0.23229465,  0.19636637,
       -0.62621973,  0.40023499,  0.47062477, -0.62863287, -0.38320842,
        0.94542783, -0.3677466 , -0.23466059, -0.2668794 ,  1.04047304,
       -0.64471287, -0.37605301,  1.26153421, -0.1057177 ,  1.26909118,
        0.93647013,  0.34078413, -0.56443002,  0.07106581,  0.94

In [ ]:

from sklearn.base import clone
from pickle import dump

In [ ]:
for model in valid_results["estimator"].values:
    # params = valid_results[valid_results["estimator"] == model]["params"][0]
    model = clone(model)
    print(model)
    model.fit(X, y)
    dump(model, open("../models/classification/{}.pkl".format(model["est"].__class__.__name__), "wb"))